# Gán Nhãn Từ Loại (Hidden Markov)

Nhóm:

19522020 - Lại Nguyễn Vĩnh Phú

19522447 - Trương Thế Trương

# library


In [27]:
import numpy as np
import random
import underthesea

DAUCAU = ",;-.?!:…"
DAUCAU_POS = ",,......"
DAUCAU_POS = ",;-.?!:…"
DAUCAU_POS = "." * len(DAUCAU)


# input and out processing
def words_to_word( words ):
	return "_".join(words)
def preprocess_sen(sen):
	sen = sen.lower()
	sen = sen.replace("%", "")
	sen = sen.replace("\n", "")

	
	for c in DAUCAU:
		sen = sen.replace(c, " "+c+" ")

	return sen
def fullOfSpace( str ):
	for c in str:
		if c not in ' \n':
			return False 
	
	return True 
def sen_to_words( sen ):


	sen = preprocess_sen( sen )

	sen_words = [ w for w in sen.split(" ") if not fullOfSpace(w) ]

	
	return sen_words
def words_and_pos_to_output_sen( sen_words, sen_pos ):
	output = ""
	n = len( sen_words)
	comma = ','
	dot = '.'

	# hide pos of ',' and '.'
	for i in range(n ):
		if i ==0 or sen_words[i] in (comma, dot):
			output += sen_words[i]
		else:
			output += ' ' + sen_words[i]
		
		if not sen_words[i] in (comma,dot):
			output += "/" + sen_pos[i].upper()



	return output
def file_to_tagged_data( path ):
	file = open(path, "r", encoding="utf8")
	text = file.readlines()

	tagged_data = []

	for line in text:
		if not fullOfSpace( line ):
			tagged_line = []

			line_words = sen_to_words( line )

			for w in line_words:
				if w in DAUCAU:
					DAUCAU_index = DAUCAU.index(w)
					tagged_line.append((w,DAUCAU_POS[DAUCAU_index]))
				else:
					s = w.split("/")
					tagged_line.append((s[0], s[1]))

			tagged_data.append( tagged_line )

	return tagged_data

#tach tu bang thuat toan Longest matching
def Longest_matching( sen, words_list):
	max_item_in_a_word = max([len(w.split('_')) for w in words_list])
	words = sen_to_words(sen)
	tokens = []
	n = len( words )
	s = 0
	while s < n :
		e = s + max_item_in_a_word
		word = words_to_word( words[s:e] )

		while ( not word in words_list ) and e>s+1:
			e -= 1
			word = words_to_word( words[s:e] )

		tokens.append( word )
		if e <= s :
			s += 1
		else:
			s = e

	return tokens

#gan nhan tu loai tuyen tinh
def find_pos_for_words( words, hidden_markov_matrix, hidden_to_observed_matrix ):
	n = len(words)
	return_pos = []
	return_pro = 1
	for i in range( n ):
		w = words[i]
		if w in DAUCAU:
			DAUCAU_index = DAUCAU.index(w)
			return_pos.append(DAUCAU_POS[DAUCAU_index])
		else:
			accept_tags_list = [ tag for tag in tags_list if tag not in DAUCAU ]
			pro = 0
			accept_pos = ""
			for pos in accept_tags_list:
				temp_pro = 1
				#calculate prepos-pos
				x_index = tags_list.index( pos )
				y_index = 0
				if i == 0:
					y_index = len(tags_list)
				else:
					y_index = tags_list.index( return_pos[i-1] )

				temp_pro *= hidden_markov_matrix[y_index,x_index]
				#calculate pos-word
				y_index = tags_list.index( pos )
				x_index = 0
				if w in words_list:
					x_index = words_list.index(w)
				else:
					x_index = len(words_list)

				temp_pro *= hidden_to_observed_matrix[y_index, x_index]

				if temp_pro > pro:
					pro = temp_pro
					accept_pos = pos
			
			return_pos.append( accept_pos )
			return_pro *= pro

	return return_pos, return_pro

#gan nhan tu loai quy hoach dong (de quy) #CANCELED
def Probality_words_pos( sen_words, sen_pos, hidden_markov_matrix, hidden_to_observed_matrix ):
	if len(sen_words) != len(sen_pos):
		return 0

	pro = 1
	for i in range( len( sen_pos)):
		x_index = tags_list.index( sen_pos[i] )
		y_index = 0
		if i == 0:
			y_index = len( tags_list)
		else:
			y_index = tags_list.index( sen_pos[i-1] )

		pro *= hidden_markov_matrix[y_index,x_index]

	for word, pos in zip( sen_words, sen_pos ):
		y_index = tags_list.index(pos)
		x_index = 0
		if word in words_list:
			x_index = words_list.index( word )
		else:
			x_index = len(words_list)

		pro *= hidden_to_observed_matrix[y_index, x_index]

	return pro
def recur_find_pos_for_words( words, hidden_markov_matrix, hidden_to_observed_matrix ):
	n = len( words )

	def recur_find_pos( pos_list,words, n, hidden_markov_matrix, hidden_to_observed_matrix):
		# stop
		if len(pos_list) == n:
			return pos_list, Probality_words_pos( words, pos_list, hidden_markov_matrix, hidden_to_observed_matrix )
		
		pro = 0
		return_pos_list = []
		word_index = len(pos_list)

		accept_tags_list = []
		if words[ word_index ] not in DAUCAU:
			accept_tags_list = [ tag for tag in tags_list if tag not in DAUCAU ]
		else:
			accept_tags_list = [words[word_index]]

		for pos in accept_tags_list:
			new_pos_list, new_pro = recur_find_pos( pos_list[:] + [pos], words, n, hidden_markov_matrix, hidden_to_observed_matrix)
			if new_pro > pro:
				return_pos_list = new_pos_list
				pro = new_pro
		
		return return_pos_list, pro
	
	return recur_find_pos([], words, n, hidden_markov_matrix, hidden_to_observed_matrix)

#viterbi
def Viterbi_find_pos_for_words( sen_words ):
	if len(sen_words) <= 0:
		return [],1
	# viterbi
	path_matrix = [[("",0) for i in range((len(sen_words)))] for i in range(len(tags_list)) ]

	for x in range( len(sen_words)):
		if x == 0:
			for y in range(len(tags_list)):
				path_matrix[y][x] = ("<s>", 0)

			accept_poses = []
			if sen_words[x] not in DAUCAU:
				accept_poses = [t for t in tags_list if t not in DAUCAU]
			else:
				accept_poses = [t for t in tags_list if t in DAUCAU]

			for pos in accept_poses:
					pro = 1
					#prepos - pos
					y_index = len(tags_list)
					x_index = tags_list.index(pos)
					pro *= hidden_markov_matrix[y_index,x_index]

					#pos - word
					y_index = tags_list.index(pos)
					x_index = 0
					if sen_words[x] not in words_list:
						x_index = len(words_list)
					else:
						x_index = words_list.index(sen_words[x])

					pro *= hidden_to_observed_matrix[y_index,x_index]

					y = tags_list.index(pos)
					path_matrix[y][x] = (path_matrix[y][x][0], pro) 

		else:
			accept_poses=[]
			if sen_words[x] not in DAUCAU:
				accept_poses = [t for t in tags_list if t not in DAUCAU]
			else:
				accept_poses = [t for t in tags_list if t in DAUCAU]

			for pos in accept_poses:
				y = tags_list.index(pos)

				#pos - word
				y_index = tags_list.index(pos)
				x_index = 0
				if sen_words[x] not in words_list:
					x_index = len(words_list)
				else:
					x_index = words_list.index(sen_words[x])

				pos_word_pro = hidden_to_observed_matrix[y_index,x_index]

				for pre_pos in tags_list:
					#prepos - pos
					pre_pos_index = tags_list.index(pre_pos)
					pro = pos_word_pro * path_matrix[pre_pos_index][x-1][1]
					y_index = tags_list.index(pre_pos)
					x_index = tags_list.index(pos)
					pro *= hidden_markov_matrix[y_index,x_index]

					if path_matrix[y][x][1] < pro:
						path_matrix[y][x] = (pre_pos, pro)
					
	#FOR SLIDE

	# find the path
	x = len(sen_words) -1
	y = 0
	pro = 0
	for i in range(len(tags_list)):
		if path_matrix[i][x][1] > pro:
			pro = path_matrix[i][x][1]
			y = i

	return_poses = [tags_list[y]]
	while x > 0:
		add_pos = path_matrix[y][x][0]
		return_poses = [add_pos] + return_poses
		y = tags_list.index(add_pos)

		x-=1

	return return_poses, pro
	
def get_words_count_from_tagged_data(tagged_data):
	words = {} 
	for line in tagged_data:
		for w, _ in line :
			if not w in words.keys():
				words[w] = 1
			else:
				words[w] += 1
	
	return words
def get_tags_count_from_tagged_data(tagged_data):
	words = {} 
	for line in tagged_data:
		for _, w in line :
			if not w in words.keys():
				words[w] = 1
			else:
				words[w] += 1
	
	return words
def get_only_tags_from_tagged_data ( tagged_data ):
	tags = []
	for line in tagged_data :
		tags_line = []
		for _, tag in line:
			tags_line.append(tag)
		
		tags.append( tags_line )
	
	return tags
def get_only_words_from_tagged_data ( tagged_data ):
	tags = []
	for line in tagged_data :
		tags_line = []
		for tag,_ in line:
			tags_line.append(tag)
		
		tags.append( tags_line )
	
	return tags
def get_tags_list_from_tagged_data( tagged_data ):
	tags = []

	for line in tagged_data:
		for _, tag in line :
			if not tag in tags:
				tags.append(tag)
	
	return tags
def get_words_list_from_tagged_data( tagged_data ):
	words = []
	for line in tagged_data:
		for w, _ in line :
			if not w in words:
				words.append(w)
	
	return words


# code


## data

In [28]:
tagged_data = file_to_tagged_data( "train_data.txt")
test_data = file_to_tagged_data( "test_data.txt")

## hidden markov


In [29]:
#  hidden markov matrix
#
#  ---------------------------
#  	   |tag0|tag1|...|tagN|
#  --------+-------------------
#  tag0    |    |    |   |    |
#  --------+----+----+---+----+
#  ...     |    |    |   |    |
#  --------+----+----+---+----+
#  tagN    |    |    |   |    |
#  --------+----+----+---+----+
#  <s>     |    |    |   |    |
#  ----------------------------


#  hidden to observed matrix
#
#  --------------------------------------+
#  	   |word0|word1|...|wordM|unknown|
#  --------+-----+-----+---+-----+-------+
#  tag0    |     |     |   |     |       |
#  --------+-----+-----+---+-----+-------+
#  ...     |     |     |   |     |       |
#  --------+-----+-----+---+-----+-------+
#  tagN    |     |     |   |     |       |
#  --------+-----+-----+---+-----+-------+

In [30]:
# get words list (dictionary), tags list, tags chains
words_list = get_words_list_from_tagged_data( tagged_data )
tags = get_only_tags_from_tagged_data( tagged_data )
tags_list = get_tags_list_from_tagged_data(tagged_data)


# create new matrix reference to hidden markov chain
n = len(tags_list)
hidden_markov_matrix = np.zeros((n+1,n))
for line in tags:
	i = 0
	n = len( line )
	while i < n:
		if i == 0:
			x_index = tags_list.index(line[i])
			y_index = len(tags_list)
		else:
			x_index = tags_list.index(line[i])
			y_index = tags_list.index(line[i-1])

		hidden_markov_matrix[y_index,x_index] += 1
		i+=1 

# create matrix reference to hidden-to-observed connect
n_tags = len(tags_list)
n_words = len(words_list)


hidden_to_observed_matrix = np.zeros((n_tags,n_words+1))
for line in tagged_data:
	for w, tag in line :
		y_index = tags_list.index(tag)
		x_index = words_list.index(w)


		hidden_to_observed_matrix[y_index,x_index] += 1



## smooth and calculate probality

In [31]:
#smooth
smooth_value = 0.1
hidden_markov_matrix = hidden_markov_matrix + smooth_value
hidden_to_observed_matrix = hidden_to_observed_matrix + smooth_value

#calculate probality
for i in range( hidden_markov_matrix.shape[0] ):
	total = np.sum( hidden_markov_matrix[i] )
	hidden_markov_matrix[i] = hidden_markov_matrix[i] / total

for i in range( hidden_to_observed_matrix.shape[0] ):
	total = np.sum( hidden_to_observed_matrix[i] )
	hidden_to_observed_matrix[i] = hidden_to_observed_matrix[i] / total


# find pos

In [32]:
# example
sen = "Tôi yêu quê hương, đất nước nhiều hoa hồng."
sen = "tết nguyên đáng là dịp ý nghĩa để tưởng nhớ tổ tiên."
sen = ",."
sen = "tôi yêu nước."
sen_words = Longest_matching( sen , words_list)


sen_pos,_ = Viterbi_find_pos_for_words(sen_words)
print(sen_pos)
words_and_pos_to_output_sen( sen_words, sen_pos )

['n', 'v', 'a', '.']


'tôi/N yêu/V nước/A.'

# accuracy

In [33]:
# acc one sen
def Accuracy_on_test_data( test_data ):
	test_pos = get_only_tags_from_tagged_data(test_data)
	test_words = get_only_words_from_tagged_data( test_data )

	total_n_pos = 0
	for sen_pos in test_pos:
		total_n_pos += len(sen_pos)

	test_sens = []
	for sen_words in test_words:
		sen = " ".join(sen_words)
		sen = sen.replace("_"," ")
		test_sens.append( sen )


	correct = 0
	for i in range( len( test_sens )):
		
		pre_sen_words = Longest_matching( test_sens[i], words_list )
		result_pos, _ = Viterbi_find_pos_for_words(pre_sen_words)

		correct += Correct_on_one_sen( pre_sen_words, result_pos, test_words[i], test_pos[i])

		print( words_and_pos_to_output_sen(test_words[i], test_pos[i]))
		print( words_and_pos_to_output_sen(pre_sen_words,result_pos))
		print( "----------------------")
	
	return correct/total_n_pos
	

def Correct_on_one_sen( pre_sen_words, pre_pos, truth_sen_words, truth_pos):
	pre_words_size = [len(pre.split("_")) for pre in pre_sen_words if pre not in DAUCAU] 
	tru_words_size = [len(tru.split("_")) for tru in truth_sen_words if tru not in DAUCAU]

	correct = 0

	for i in range( len(tru_words_size) ):
		tru_start = sum( tru_words_size[:i] )
		j=0
		pre_start = 0
		while pre_start < tru_start:
			j+=1
			pre_start = sum( pre_words_size[:j])

		if pre_start == tru_start:
			if pre_pos[j] == truth_pos[i] and pre_sen_words[j] == truth_sen_words[i] and (truth_sen_words[i] not in DAUCAU): 
				correct += 1

	return correct


In [34]:
test_words_list = get_words_list_from_tagged_data( test_data )
unknown_count = 0
for w in test_words_list:
	if w not in words_list:
		unknown_count += 1

print( len(test_words_list ), unknown_count )

83 3


In [35]:
Accuracy_on_test_data( test_data )

hôm_nay/N gia_đình/N hạnh_phúc/A.
hôm_nay/N gia_đình/N hạnh_phúc/A.
----------------------
anh/N chỉ/R thích/V em/N thôi/I.
anh/N chỉ/R thích/V em/N thôi/A.
----------------------
tôi/N đang/R chơi/V bóng_đá/N.
tôi/N đang/R chơi/V bóng_đá/N.
----------------------
tôi/N yêu/V hoa_hồng/N.
tôi/N yêu/V hoa_hồng/N.
----------------------
tôi/N thường_xuyên/A ăn/V dưa_hấu/N.
tôi/N thường_xuyên/A ăn/V dưa_hấu/N.
----------------------
học_tập/V để/C bản_thân/N không/R thua_thiệt/V với/C bạn_bè/N.
học_tập/V để/C bản_thân/N không/R thua_thiệt/V với/C bạn_bè/N.
----------------------
tôi/N ghét/V chơi/V bóng/N.
tôi/N ghét/R chơi/V bóng/A.
----------------------
ai/P cũng/R tắm/V.
ai/P cũng/R tắm/V.
----------------------
tết_nguyên_đán/N là/V ngày/N đoàn_tụ/V gia_đình/N.
tết_nguyên_đán/N là/V ngày/N đoàn_tụ/V gia_đình/N.
----------------------
đây/P là/V khoảng/N thời_gian/N thiêng_liêng/A của/E người/N việt_nam/N.
đây/P là/V khoảng/N thời_gian/N thiêng_liêng/A của/E người/N việt_nam/N.
-------

0.7952755905511811

In [36]:
Accuracy_on_test_data( tagged_data )

gia_đình/N là/V tổ_ấm/N hạnh_phúc/A.
gia_đình/N là/V tổ_ấm/N hạnh_phúc/A.
----------------------
hôm_nay/N trời/N đẹp/A quá/R.
hôm_nay/N trời/N đẹp/A quá/R.
----------------------
tôi/N thua_thiệt/V với/C bạn_bè/N.
tôi/N thua_thiệt/V với/C bạn_bè/N.
----------------------
ba/N là/V người/N đàn_ông/N trụ_cột/N trong/C gia_đình/N.
ba/N là/V người/N đàn_ông/N trụ_cột/N trong/C gia_đình/N.
----------------------
bố_mẹ/N là/V tuyệt_vời/A nhất/R đối_với/C chúng_ta/P.
bố_mẹ/N là/V tuyệt_vời/A nhất/R đối_với/C chúng_ta/P.
----------------------
hoa_hồng/N là/V một/M loài/N hoa/N đẹp/A.
hoa_hồng/N là/V một/M loài/N hoa/N đẹp/A.
----------------------
tôi/P yêu/V quê_hương/N, đất_nước/N.
tôi/N yêu/V quê_hương/N, đất_nước/N.
----------------------
tiếng/N việt/N cực_kỳ/R phong_phú/A, có_thể/R nói/V là/V phức_tạp/A trong/C cấu_tạo/N và/C cả/P ngữ_nghĩa/N.
tiếng/N việt/N cực_kỳ/R phong_phú/A, có_thể/R nói/V là/V phức_tạp/A trong/C cấu_tạo/N và/C cả/P ngữ_nghĩa/I.
----------------------
ai/P cũng/R 

0.7791842475386779

# compare longest matching

In [37]:
def compare_2_sen_words( predict , truth):
	pre_words_size = [len(pre.split("_")) for pre in predict if pre not in DAUCAU] 
	tru_words_size = [len(tru.split("_")) for tru in truth if tru not in DAUCAU]

	count = 0
	start_pre = 0
	index_pre = 0
	while start_pre < sum(pre_words_size):
		start_tru = 0
		index_tru = 0
		while start_tru<start_pre:
			start_tru+=tru_words_size[index_tru]
			index_tru += 1
		
		if start_tru == start_pre and pre_words_size[index_pre] == tru_words_size[index_tru]:
			count += 1
		
		start_pre += pre_words_size[index_pre]
		index_pre += 1

	return count


compare_2_sen_words( sen_to_words("ông bà khác ông và bà."), sen_to_words("ông_bà,,, khác ông và bà"))



4

In [38]:
# Longest matching
test_sen_words = get_only_words_from_tagged_data( test_data )
words_count = 0
correct = 0
for sen_words in test_sen_words:
	words_count += len(sen_words)

	sen = " ".join(sen_words)
	sen = sen.replace("_", " ")

	Longest_matching_words = Longest_matching(sen, words_list)
	correct += compare_2_sen_words( Longest_matching_words, sen_words)

correct / words_count

0.84251968503937

In [39]:
# underthesea
from underthesea import word_tokenize
# Longest matching
test_sen_words = get_only_words_from_tagged_data( test_data )
words_count = 0
correct = 0
for sen_words in test_sen_words:
	words_count += len(sen_words)

	sen = " ".join(sen_words)
	sen = sen.replace("_", " ")

	underthesea_words = word_tokenize( sen )
	for i in range( len(underthesea_words )):
		underthesea_words[i] = underthesea_words[i].replace(" ","_")
	print(underthesea_words)
	correct += compare_2_sen_words( underthesea_words, sen_words)

correct / words_count


['hôm_nay', 'gia_đình', 'hạnh_phúc', '.']
['anh', 'chỉ', 'thích', 'em', 'thôi', '.']
['tôi', 'đang', 'chơi', 'bóng_đá', '.']
['tôi', 'yêu', 'hoa_hồng', '.']
['tôi', 'thường_xuyên', 'ăn', 'dưa_hấu', '.']
['học_tập', 'để', 'bản_thân', 'không', 'thua_thiệt', 'với', 'bạn_bè', '.']
['tôi', 'ghét', 'chơi', 'bóng', '.']
['ai', 'cũng', 'tắm', '.']
['tết', 'nguyên_đán', 'là', 'ngày', 'đoàn_tụ', 'gia_đình', '.']
['đây', 'là', 'khoảng', 'thời_gian', 'thiêng_liêng', 'của', 'người', 'việt', 'nam', '.']
['mọi', 'người', 'trong', 'gia_đình', 'chúc', 'tết', 'ông_bà', '.']
['ngày', 'để', 'tưởng_nhớ', 'những', 'người_thân', 'đã', 'mất', '.']
['thầy_thuốc', 'không', 'bao_giờ', 'lựa_chọn', 'món', 'ăn', 'ngon', '.']
['trong', 'dịp', 'tết', 'dương_lịch', ',', 'người', 'việt', 'sắm_sửa', 'hai_mươi', 'mốt', 'mâm', 'ngũ_quả', '.']
['gia_đình', 'rất', 'quan_trọng', 'đối_với', 'mỗi', 'con_người', '.']
['ngày', 'gia_đình', 'dọn_dẹp', 'là', 'ngày', 'mười', 'chín', 'tháng', 'một', 'hằng', 'năm', '.']
['nén', 'hương

0.8031496062992126